In [42]:
import ecdsa
import hashlib
import random
accounts = { }
list_sk = []

for i in range(5):
    sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    list_sk.append(sk)
    pk = sk.verifying_key
    accounts[pk.to_string().hex()[:40]] = random.randint(0,100)

account_adress = list(accounts.keys())

In [43]:
class State :
    def __init__(self, accounts) :
        self.acc = accounts.keys()
        self.amo = accounts.values()
        self.accounts = accounts
        self.smartContracts = {}
    
    def printAccounts(self):
        print(self.accounts)
    def printSmartContracts(self):
        print(self.smartContracts)
        
#Test
st1 = State(accounts)

st1.printAccounts()

{'de8a596a7033d760d53f97941f84a32bcdf08795': 87, 'f1aa9888931a8d2f36ca582d58281537c39a8610': 19, 'a6382845263a5a1879529e51b8ce073f23016dd5': 7, '8683d259fc13e0b3fe4f14b7661129412ec6b839': 64, '704f079f775bb558adca8426213c8173513965ca': 22}


In [44]:
class User:
    '''
    Containing serete key and public key and adress of a user
    Attributes:
     - secrete key
     - public key
     - adress
    '''
    def __init__(self, secrete_key, public_key, adress):
        self.sk = secrete_key
        self.pk = public_key
        self.adr = adress
    
class Transaction : 
    def __init__(self, sender, receiver, amount, typeTransaction, data, signature = None ): 
        '''
        Initial a transaction

        Arguments:
        - sender{user}: information of sender (sk and pk and adr)
        - receiver{hex}: adress of the receiver
        - amount{int}: the amount of coin sent
        - signature{bytes}: optinal parameter
        '''
        self.sender = sender        
        self.receiver = receiver    
        self.amount = amount        
        self.signature = signature  
        self.typeTransaction = typeTransaction #new one
        self.data = data   # new attributte
        
    def buildSig(self, sk):
        '''
        Build a signature for a transaction 
        
        Arguments:
        - sk{ecdsa.keys.SigningKey}: secret key of the sender

        Return:
        - signature{bytes}: signature of the transaction
        '''
        #reciever + amount
        txData = str(self.receiver) + str(self.amount)

        #convert Data from string to bytes 
        txDataBytes = txData.encode()
        
        #the hash of Data
        self.hashOfData = hashlib.sha3_256(txDataBytes).digest() # output in bytes
        
        #create signature for this transaction
        self.signature = sk.sign(self.hashOfData) #in bytes 
          
        return self.signature
             
    def verifySig(self, signature):
        '''
        Verify the signature of the transaction

        Arguments:
        - signature{bytes}

        Return:
        -True: If the signature is correct
        -False: Otherwise
        '''
        try: 
            assert self.sender.pk.verify(signature, self.hashOfData)
            return True
        except:
            return False
            
    def recoverSenderAddressCandidates(self, signature):
        '''
        Recover the adress of the sender from its signature
        Arguments:
        - signature{bytes}: the signature of the transaction

        Return:
        - h[0],h[1] {hex}: List two potential values for the adress of the sender
        '''
        curve = ecdsa.curves.SECP256k1
        h = ecdsa.VerifyingKey.from_public_key_recovery(
		signature=signature, data=self.hashOfData, curve=curve, sigdecode=ecdsa.util.sigdecode_string)
        
        #return 2 potentials adress 
        return [h[0].to_string().hex()[0:40],h[1].to_string().hex()[0:40]]    
        
    def verifyTransaction(self, state): 
        '''
        Verify the transaction is satisfied by the conditions

        Arguments:
        - state{State}: the state of system

        Return:
        -True: If the transaction is correct
        -False: Otherwise
        '''
        #verify account exist in system
        h = []
        h = self.recoverSenderAddressCandidates(self.buildSig(self.sender.sk))
        if h[0] in state.acc:
            self.sender_account = h[0]
        elif h[1] in state.acc:
            self.sender_account = h[1]
        else:
            return False
        if self.receiver not in state.acc:
            return False
        
        #verify signature is correct 
        
        if self.verifySig(self.buildSig(self.sender.sk)) == False:
            return False
        
        #verify sender's account has enough money 
        
        if state.accounts[self.sender_account] < self.amount:
            return False
     
        return True   
    
    def updateState(self, state):
        '''
        After a succes transaction, the state of system need to update

        Arguments:
        - state{State}: the state of system

        Return:
        - state{State}: new state of system

        '''
        if self.verifyTransaction(state) == True:
            state.accounts[self.sender_account] -= self.amount
            state.accounts[self.receiver] += self.amount
        

    def infors(self):
        '''
        Given info of the transaction

        Return:
        - sender{hex}: adress of the sender
        - receiver{hex}: adress of the receiver
        - amount{int}: the amount of coint sent
        '''
        return{
            'sender': self.sender.pk.to_string().hex()[:40],
            'receiver': self.receiver,
            'amount': self.amount
        }



In [45]:
#TEST

# We already created a list 5 accounts include secrete key and public key
list_sk
# Now we create a profil for these 5 users
user = []
for i in range(5):
    user.append(User(list_sk[i], list_sk[i].verifying_key, list_sk[i].verifying_key.to_string().hex()[0:40]))

# Now we have the state of system
state = State(accounts)


In [46]:
class Block : 
    def __init__(self, index, previous_hash, transactions = None) : 
        '''
        Inital a block

        Arguments:
        -index{int}: index of the block
        -previous_hash{bytes}: the hash of prevous block
        -transaction{Transaction}: optional patrameter
        '''
        self.index = index
        self.previous_hash = previous_hash
        self.transactions = []
        self.current_hash = self.computeHash()
    def addTransaction(self, transaction):
        '''
        Add a transaction to the block

        Arguments:
        -transaction{Transaction}: new transaction want to add

        Return:
        -new state of block after adding new transaction (change in current_hash)

        '''
        self.transactions.append(transaction)
        self.current_hash = self.computeHash()
    def computeHash(self):
        '''
        Compute the hash of block

        Arguments:
        -None

        Return:
        -txHsah{bytes}: computed hashe of block
        '''
        
        tx = str(self.index) + str(self.transactions) + str(self.previous_hash)
        txBytes = tx.encode()
        txHash = hashlib.sha3_256(txBytes).digest()
        return txHash

    def verifyBlock(self, state):
        '''
        Verify the block is correct

        Arguments:
        -state{State}: the state of system

        Return:
        -True:if the block is correct
        -False: if the block is not correct
        '''
        #check if the hash of the block is correct
        if self.current_hash != self.computeHash():
            return False
        #check if all transactions in the block are valid
        for transaction in self.transactions:
            if not transaction.verifyTransaction(state):
                return False
        return True
    
    def infors(self):
        return{
            "Index": self.index,
            "Transactions": [t.infors() for t in self.transactions],
            "Previous Hash": self.previous_hash,
            "Hash": self.current_hash
        }
    @staticmethod
    def create_genesis_block():
        '''
        Create the genesis block
    
        '''
        return Block(0,'0', [])
    @staticmethod
    def next_block(last_block, transactions):
        '''
        Create the next block

        Arguments:
        -last_block{Block}: the last block of the chain
        -transactions{Transaction}: list of transactions in the new block
        '''
        index = last_block.index +1
        previous_hash = last_block.current_hash
        return Block(index, previous_hash, transactions)




In [47]:
from __future__ import print_function
import random
import sys
import time
from pysyncobj import SyncObj, replicated
from pysyncobj.batteries import ReplList

class Blockchain(ReplList):
    def __init__(self, state):
        super(Blockchain, self).__init__()
        self.state = state
        self.n = 0   # check if the blockchain is empty
        
        
    def addBlock(self, newblock):
        self.append(newblock)

    def append(self, newblock):
        if self.n == 0:
            self.n += 1
            self._ReplList__data.append(Block.create_genesis_block())
            
        if newblock.verifyBlock(state) == False: 
            print('Invalid block ! - blocks index: ', newblock.index)
            return False
        if newblock.previous_hash != self._ReplList__data[-1].current_hash:
            print('The new block is not connected to the chain - blocks index: ', newblock.index)
            return False
        
        
        self._ReplList__data.append(newblock)
        
        for transaction in newblock.transactions:
            if transaction.typeTransaction == 0:
                transaction.updateState(self.state)
                print("Transaction type 0: Updated state")
            elif transaction.typeTransaction == 1:
                        
                #for deploying smart contract, create a new contract object and add the result to the state
                smartContract = {}
                smartContract[transaction.data['address']] = transaction.data
                self.state.smartContracts[transaction.data['address']]= smartContract
                print("Transaction type 1: Deployed smart contract")
                        
                transaction.updateState(self.state)
            elif transaction.typeTransaction == 2:
                #for calling smart contract, execute the smart contract and add it to the state
                contract = self.state.smartContracts[transaction.data['address']]
                if contract is not None:
                    #get the values of the key 'setup' firstly then 'addDouble' to execute the smart contract
                    for i in contract.values():                                
                        for key, value in i.items():
                            if key == 'setup':
                                setup_data = value
                            if key == transaction.data['scriptname']:
                                exec(setup_data + '\n' + value.format(transaction.data['param1']))
                    transaction.updateState(self.state)
                    print("Transaction type 2: Called smart contract")

        return True

       


In [49]:
#TEST
data1 = {
"address" : "6eb59616a69838dbc14fc0d666e16" , # the address of the contract
"owner" : user[0].adr, # the adress of the contract owner
"setup" : "value=0\nprint('value=',value)", # the script that must be launched at the deployement
"addDouble" : "value = value + 2*{}\nprint('value=',value)", # a script of the contract
"fees" : 1 # fees to be paid to the owner
}

data2 = {"address" : "6eb59616a69838dbc14fc0d666e16" , "scriptname" : "addDouble", "param1" : 4}

trans1 = Transaction(user[4], user[1].adr, 1, typeTransaction=0, data=None)
trans2 = Transaction(user[0], user[1].adr, -1, typeTransaction=0, data=None)
trans3 = Transaction(user[3], user[2].adr, 0, typeTransaction=0, data=None)
trans4 = Transaction(user[1], user[2].adr, 1, typeTransaction=0, data=None)
trans5 = Transaction(user[0], user[1].adr, 0, 1, data1)
trans6 = Transaction(user[1], user[0].adr, 0, 2, data2)
#Create a false transaction
False_transaction = Transaction(user[0],'ccekcmklekcma234mfkre4323md2', 9,typeTransaction=0, data=None)  # transation not exist in system
state = State(accounts)

def test_addBlock():
    block_chain = Blockchain(state)

    gens_block = Block.create_genesis_block()
    block1 = Block(1, gens_block.computeHash())

    block1.addTransaction(trans1)
    block1.addTransaction(trans2)
    block_chain.addBlock(block1)

    block2 = Block(2, block1.computeHash())
    block_chain.addBlock(block2)
    
    block3 = Block(3, block1.current_hash)        #create the thrid block connect to block 1
    block3.addTransaction(trans2)
    block_chain.addBlock(block3)

    
    block4 = Block(4, block2.current_hash)       # create the fouth block
    block4.addTransaction(False_transaction)
    
    
    block_chain.addBlock(block4)
    
    print('length of blockchain: ', len(block_chain))


if __name__ == '__main__':
    test_addBlock()

    

Transaction type 0: Updated state
Transaction type 0: Updated state
The new block is not connected to the chain - blocks index:  3
Invalid block ! - blocks index:  4
length of blockchain:  3
